In [1]:
from PIL import Image
import numpy as np

In [8]:
image = 'assets/nermal.jpg'

img = Image.open(image).convert('RGB')

pixels = np.array(img)

display(pixels)

pixelsFlattened = pixels.flatten()
display(pixelsFlattened)

array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], shape=(360, 480, 3), dtype=uint8)

array([0, 0, 0, ..., 0, 0, 0], shape=(518400,), dtype=uint8)

In [ ]:
lsbArray = pixelsFlattened & 1 #this will isolate the least significant bit as we 

#Now we can analyze the array of least significant bits to get a string of bits

First 100 bits of LSB: [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [16]:
groupedLSB = []
binary = ""
for idx in range (0, len(lsbArray)):
    binary += str(lsbArray[idx])
    if (idx % 8 == 7):
        groupedLSB.append(binary)
        binary = ""

display(groupedLSB)

['00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',
 '00000000',

In [19]:
outputString = ""
for binString in groupedLSB:
    if (len(binString) == 8):
        outputString += chr(int(binString,2))

print(outputString)

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        